In [11]:
# !pip install -q langchain==0.1.9
# !pip install -q openai==1.13.3

In [3]:
from keys import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key


In [6]:
from langchain_community.llms import OpenAI
llm = OpenAI(temperature=0.6)
name = llm.invoke("I want to open a restaurant for Chinese food. Suggest a fancy name for this.")
print(name)



Golden Dragon Palace 


In [7]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

prompt_template_name.format(cuisine="Mexican")

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

In [8]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)
name_chain.run(cuisine="Mexican")

/home/chong-u/mambaforge/envs/codebasics-langchain-crash-course/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"La Cantina de Sabores" (The Flavor Cantina)'

## Sequential Chains

In [9]:
prompt_template_items = PromptTemplate(
    input_variables = ["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list.",
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [10]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response = chain.run("Indian")
print(response)



1. Chicken Tikka Masala
2. Vegetable Samosas
3. Lamb Vindaloo
4. Palak Paneer
5. Tandoori Chicken
6. Garlic Naan
7. Chana Masala
8. Aloo Gobi
9. Butter Chicken
10. Mango Lassi 



## Sequential Chain

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template = "Suggest some menu items for {restaurant_name}. Return the menu items as a single, comma separated string with no additional preamble.",
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")



In [13]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain], 
    input_variables=["cuisine"], 
    output_variables = ["restaurant_name", "menu_items"]
)

chain.invoke({"cuisine": "Singaporean"})

{'cuisine': 'Singaporean',
 'restaurant_name': '\n"Straits Flavors"',
 'menu_items': '\n\nSambal Shrimp, Roti Canai, Nasi Lemak, Beef Rendang, Satay Skewers, Hainanese Chicken Rice, Char Kway Teow, Laksa Soup, Rojak Salad, Curry Puffs'}

## Agents

In [20]:
!pip install -q wikipedia
!pip install -q numexpr # for llm-math

In [21]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [22]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)